In [204]:
import numpy as np
import pandas as pd
import pandas_profiling as pp

import matplotlib.pyplot as plt
%matplotlib inline

In [205]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submit = pd.read_csv('Sample_Submission.csv')

In [206]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [207]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [208]:
sample_submit.head()

,User_ID,Product_ID,Purchase


In [209]:
train.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Purchase'],
      dtype='object')

In [210]:
test.columns

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3'],
      dtype='object')

In [211]:
train.columns.difference(test.columns)

Index(['Purchase'], dtype='object')

In [212]:
print("train size :",train.shape)
print("test size :",test.shape)

train size : (550068, 12)
test size : (233599, 11)


In [213]:
dataf = pd.concat([train,test], axis=0, sort=False)

In [214]:
dataf.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370.0
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200.0
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422.0
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057.0
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969.0


In [215]:
duplicate_bool = dataf.duplicated(subset=dataf.columns, keep='first')
duplicate = dataf.loc[duplicate_bool == True]
duplicate   # returns no rows - means no duplicates

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase


In [216]:
dataf.Gender.replace(['M','F'],[0,1], inplace=True)

In [217]:
dataf.Age.replace(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],[1,2,3,4,5,6,7], inplace=True)

In [218]:
dataf.City_Category.replace(['A', 'C', 'B'],[1,2,3],inplace=True)

In [219]:
dataf.replace(['2', '4+', '3', '1', '0'],[1,2,3,4,5], inplace=True)

In [220]:
def conv_dict(s):
    ret = {}
    for i in range(len(s)):
        #ret[i] = s[i]
        ret[s[i]] = i
    return ret

# 1. Get the Unique Product_ID
prod_u = dataf.Product_ID.unique()

# 2. Pass this to generate Product_ID dictionary
prod_dict = conv_dict(prod_u)

In [221]:
# 3. Using the above dictionary convert Product_ID value from string to int
dataf['Product_ID']=dataf['Product_ID'].apply(lambda x: prod_dict[x])

In [222]:
dataf.Product_Category_2.fillna(0, inplace=True)

In [223]:
dataf.Product_Category_3.fillna(0, inplace=True)

In [ ]:
# Droping the user_id column
dataf.drop(['User_ID'], axis=1, inplace=True)

In [231]:
train_modified = dataf[dataf.Purchase.notnull()]

In [237]:
test_modified = dataf[dataf.Purchase.isnull()]
test_modified.drop(columns=['Purchase'],inplace=True)

C:\Users\sensundararajan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [239]:
print("train_modified size :",train_modified.shape)
print("test_modified size :",test_modified.shape)

train_modified size : (550068, 11)
test_modified size : (233599, 10)


In [240]:
train_modified.to_csv("train_ready_for_model.csv",index=False)

In [241]:
test_modified.to_csv("test_ready_for_model.csv",index=False)